# Data Concatenation / Initial Cleaning
Used to combine the disparate jsonlines files from [OpenBeta](https://github.com/OpenBeta/climbing-data/tree/next) into one large csv of only non-bouldering routes. This includes transforming the column types to be easier to work with, and creation of numeric grade columns, a date established column, and if the grade has a plus/minus columns. The final dataset is saved to a csv.

In [2]:
import pandas as pd
import numpy as np
import os
import json
import re
import contractions
import spacy
import datefinder
from sklearn.model_selection import train_test_split

#so no data is hidden
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [3]:
route_file_list = []

for dirpath, subdirs, filenames in os.walk('./open-beta-routes/'):
    route_file_list.extend([os.path.join(dirpath, name) for name in filenames if 'routes' in name])

In [4]:
#https://sundararamanp.medium.com/a-relatively-faster-approach-for-reading-json-lines-file-into-pandas-dataframe-90b57353fd38
df_list = []
for file in route_file_list:
    with open(file, 'r') as f:
        lines = f.read().splitlines()
    df_inter = pd.DataFrame(lines)
    df_inter.columns = ['json_element']
    df_list.append(pd.json_normalize(df_inter['json_element'].apply(json.loads)))

df_final = pd.concat(df_list)

In [5]:
df_final = df_final.reset_index(drop=True)
df_final.head()

,route_name,safety,fa,description,location,protection,mp_sector_id,mp_route_id,grade.YDS,grade.French,grade.Ewbanks,grade.UIAA,grade.ZA,grade.British,type.trad,metadata.left_right_seq,metadata.parent_lnglat,metadata.parent_sector,metadata.mp_route_id,metadata.mp_sector_id,metadata.mp_path,grade.Font,type.boulder,type.sport,type.tr,type.alpine,grade.yds_aid,type.aid,type.snow,type.mixed,type.ice
0,Gravel Pit,,Jason Milford/ Matt Schutz Spring 2020,"[Goes up slab on bolts to steep corner on gear. Leaves the corner via steep roof traverse crack,...",,"[Chains on top, can lower off easy. Pro to 3"" and a few draws for bolts.]",119029240,119029258,5.12b/c,7b+,27,IX-,27,E6 6b,True,999999,"[-118.20202, 35.4963]",Beerhorn Crag,119029258,119029240,Sierra Eastside|Beerhorn Crag,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Random Impulse,,"""Unknown"" or",[Some fun moves broken up by a few scree filled ledges and a big bush. Crux comes half way up wh...,[25 feet to the right of Deep Springs Education.],[A small assortment of cams and maybe a nut or two],119100232,119101118,5.7,5a,15,V+,13,MVS 4b,True,1,"[-118.13831, 37.3129]",Westgard Pass East Side (Hwy 168),119101118,119100232,Sierra Eastside|Westgard Pass East Side (Hwy 168),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,The Tick Wall,,"7, July 2020","[Bouldering. Approximately 14’ tall and 20’ or so wide that I have cleared and cleaned to date, ...",[Park at Sycamore Creek bridge and walk upstream along the old asphalt road for approximately 30...,[None. Bring your own pad.],119181845,119181945,V-easy,NaN,NaN,NaN,NaN,NaN,NaN,999999,"[-119.27525, 36.91821]",Sycamore Creek Bouldering,119181945,119181845,Western Sierra|Sycamore Creek Bouldering,3,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Orange Crush,,"Wade Griffith, Sterling Killion, Scott Williams",[Pretty cool orange arete that sports some interesting climbing. Crimpy edges start you off cli...,[The route is located on the far southern shoulder of Yeti dome which is actually located on the...,[7 QD's],105817198,105817201,5.11b/c,6c+,23,VIII-,24,E4 6a,NaN,0,"[-119.3094, 37.1667]",Yeti Dome (aka Musick Mountain),105817201,105817198,Western Sierra|Yeti Dome (aka Musick Mountain),NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Wimovi Wonder Winos,,Kroll and McHam,[Climb the most open looking slab on the more eastern aspect of the wall closer to the road. Go ...,"[Upper right of the wall, more facing the road.]",[Bolts],113627837,118979787,5.10-,6a,18,VI+,18,E1 5a,NaN,12,"[-119.31008, 37.14469]",Wimovi Pluton,118979787,113627837,Western Sierra|Wimovi Pluton,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211123 entries, 0 to 211122
Data columns (total 31 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   route_name               211123 non-null  object
 1   safety                   211123 non-null  object
 2   fa                       211123 non-null  object
 3   description              211123 non-null  object
 4   location                 211123 non-null  object
 5   protection               211123 non-null  object
 6   mp_sector_id             36604 non-null   object
 7   mp_route_id              36604 non-null   object
 8   grade.YDS                206795 non-null  object
 9   grade.French             130665 non-null  object
 10  grade.Ewbanks            130665 non-null  object
 11  grade.UIAA               130665 non-null  object
 12  grade.ZA                 130665 non-null  object
 13  grade.British            130665 non-null  object
 14  type.trad           

### Drop Unnecessary Rows/Columns

In [6]:
#slice to get rid of bouldering routes + drop boulder column - wrong grading system
df_final = df_final[df_final['type.boulder'].isna()].drop(columns=['type.boulder'])

In [7]:
#slice to get rid of aid routes + drop aid column- wrong grading system
df_final = df_final[df_final['grade.yds_aid'].isna()].drop(columns=['type.aid'])

In [8]:
df_final.isna().sum()

route_name                      0
safety                          0
fa                              0
description                     0
location                        0
protection                      0
mp_sector_id               108689
mp_route_id                108689
grade.YDS                    3976
grade.French                 3976
grade.Ewbanks                3976
grade.UIAA                   3976
grade.ZA                     3976
grade.British                3976
type.trad                   70609
metadata.left_right_seq         0
metadata.parent_lnglat          0
metadata.parent_sector          0
metadata.mp_route_id            0
metadata.mp_sector_id           0
metadata.mp_path                0
grade.Font                 132274
type.sport                  68993
type.tr                    112794
type.alpine                126456
grade.yds_aid              132274
type.snow                  131597
type.mixed                 130918
type.ice                   128955
dtype: int64

A lot of this information can be dropped. I would be tempted to use the threshold parameter of dropna, but there are a number of columns with many null values that still need to be explored simply because the route type likely has a large affect on the grade. Columns to drop:
- mp_sector_id: Missing for many rows, and the full sector ids are stored in the metadata.mp_sector_id
- mp_route_id: Same story as sector id
- any grade not grade.YDS: these grades all have a direct mapping, and YDS is the one we'll be working in
- metadata.left_right_seq: this tells if the routes are listed from left to right on the area page.

The more esoteric type columns, such as aid, snow, mixed, and ice will likely be dropped, as well as alpine and potentially tr (top rope). I would like to explore those in the more formal EDA though.

In [9]:
cols_to_drop = [x for x in df_final.columns if "grade" in x and not "YDS" in x]
cols_to_drop.extend(['mp_route_id', 'mp_sector_id', 'metadata.left_right_seq'])
df_final = df_final.drop(columns=cols_to_drop)

### Clean up Column Dtypes
Next I would like to amend the dtypes of the columns. The NLP features are in lists, which would be better as just the strings those lists contain, combined into one feature. The type indicator columns should be one hot encoded rather than having nans. The rest are already strings and can remain that way for now.

In [10]:
for col in df_final.columns:
    print(f"{col} dtype: {type(df_final.loc[1, col])}")

route_name dtype: <class 'str'>
safety dtype: <class 'str'>
fa dtype: <class 'str'>
description dtype: <class 'list'>
location dtype: <class 'list'>
protection dtype: <class 'list'>
grade.YDS dtype: <class 'str'>
type.trad dtype: <class 'bool'>
metadata.parent_lnglat dtype: <class 'list'>
metadata.parent_sector dtype: <class 'str'>
metadata.mp_route_id dtype: <class 'str'>
metadata.mp_sector_id dtype: <class 'str'>
metadata.mp_path dtype: <class 'str'>
type.sport dtype: <class 'float'>
type.tr dtype: <class 'float'>
type.alpine dtype: <class 'float'>
type.snow dtype: <class 'float'>
type.mixed dtype: <class 'float'>
type.ice dtype: <class 'float'>


In [11]:
#binarize climbing type columns
type_cols = [x for x in df_final.columns if "type" in x]
for col in type_cols:
    df_final[col] = df_final[col].apply(lambda x: 0 if type(x) != 'str' and np.isnan(x) else 1)

In [12]:
#take the nlp features out of their lists
nlp_cols = ['description', 'location', 'protection']
for col in nlp_cols:
    df_final[col] = df_final[col].apply(lambda x: " ".join(x))

#### Text Cleaning with spaCy
spaCy is a powerful NLP tool which can be used for various text data cleaning and inspection. Here we will use it to remove stopwords, along with other regular expressions and string operations for cleaning.

In [13]:
#load in model - don't need the disabled pipelines
nlp = spacy.load('en_core_web_lg',disable=['ner'])

In [14]:
#combine the nlp features into one all-lowercase column, expand contractions,  remove non-letters
df_final['text_combined'] = df_final.apply(lambda x: " ".join([str(x['description']), 
                                                           str(x['location']), 
                                                           str(x['protection'])]).lower(), 
                                       axis=1)
#drop np.nans, remove str np.nans, drop any that are only whitespace
df_final = df_final.dropna(subset='text_combined')
df_final['text_combined'] = df_final['text_combined'].apply(lambda x: x.replace("np.nan", ""))
df_final = df_final.drop(df_final[df_final['text_combined'].str.isspace()].index)

#expand contractions, remove non-letters
df_final['text_combined'] = df_final['text_combined'].map(contractions.fix)
df_final['text_combined'] = df_final['text_combined'].apply(lambda x: x.replace("'s ", " "))
df_final['text_combined'] = df_final['text_combined'].apply(lambda x: re.sub(r'[^a-z\s]', ' ', x.lower()))

#lemmatization with stopword and extra whitespace removal - takes awhile to run
df_final['lemmatized_text_combined'] = df_final['text_combined'].apply(lambda x: ' '.join([token.lemma_ for token in nlp(x) if not token.is_stop and not token.is_space]))


In [15]:
#final check for nan/empty values
nans_to_drop = df_final[(df_final['lemmatized_text_combined'] == '') | (df_final['lemmatized_text_combined'] == 'nan')].index
df_final = df_final.drop(nans_to_drop)

### Transforming YDS Grades
To compare grades, we need to do some processing to make the grades more computationally legible. There are a few components to the grades ([adapted with help from this article](https://www.sportrock.com/post/understanding-climbing-grades)):
- Class: 5
    - The class indicates the difficulty of the terrain, with 1 being flat land. While 4th class terrain may require ropes, all climbs are at least 5th class.
- Difficulty: .0-.15
    - Most true climbing starts at a grade of 5.4 or 5.5, with lower grades being seen mostly as scrambles.
- Letter: a-d
    - The letter grade indicates the sub-difficulty of the climb within the number difficulty. Letter grades are only used for 5.10 climbs and up. This is how we distinguish between easy and hard climbs within a grade that do not quite fall into the neighboring grades.
- +/-:
    - The + and - after the grade are similar to the letter system, but less precise. These may be used on any grade of climb, and are never used in conjunction with letters. A + on older routes (think pre-1980) can often be construed to mean that the real feel of the route is much more difficult than the given grade. In general, a + can be thought of as similar to the letter grade c/d, and a - can be thought of as similar to the letter grade a/b
- Risk Rating: PG, PG13, R, X
    - This conveys how run out a route is, and the true physical danger of the route if the lead climber were to take a fall. A climb with an X risk rating is a climb you -do not- want to fall on.
    
Our goal is to categorize 5th class climbs by grade. Our main approach will be as a classification problem. If we leave the target variable, `grade.YDS` as-is, there are 71 classes, some with very few rows per class as shown below:

In [23]:
df_final['grade.YDS'].value_counts()

5.9        11387
5.8        10392
5.7         8460
5.10a       7703
5.10b       5671
5.11a       5527
5.6         5424
5.10c       4932
5.12a       4328
5.11b       4155
5.10        4015
5.9+        3948
5.10d       3931
5.11c       3538
5.11d       2877
5.12b       2678
5.10+       2657
5.11        2643
5.8+        2556
5.5         2538
5.12c       1881
5.4         1722
5.10-       1721
5.11+       1712
5.11-       1537
5.7+        1493
5.13a       1376
5.12d       1229
5.9-        1221
5.10a/b     1120
5.10b/c     1118
5.12        1018
5.12-        871
5.11b/c      862
5.13b        771
5.8-         767
5.11a/b      732
5.3          731
5.10c/d      709
5.12a/b      579
5.11c/d      531
5.12+        514
5.12b/c      473
5.13c        411
5.2          396
5.13d        270
5.12c/d      258
5.13-        246
5.13         226
5.14a        223
5.0          166
5.13a/b      135
5.1          118
5.13b/c      113
5.13+        109
5.14b         77
5.13c/d       53
5.14-         38
5.14c         

In initial modeling, these extreme class imbalances, combined with the number of classes available and small stature of our dataset, led me to decide to reduce the number of classes to just the base 16 (5.0-5.15). 

In [24]:
def fully_reduce_grade(grade):
    """
    Takes in a YDS grade and returns just the truncated number value
    Does not take into account any letter grade or +/-
    Returns the number difficulty 0-15 to be used as a label
    """
    #reduce to difficulty grade, the 5. is not informational
    grade = grade.split(".")[1]
    
    #check for +/-/letters
    if grade[-1] not in "1234567890":
        if "/" in grade:
            return grade[:-3]
        return grade[:-1]
    
    return int(grade)

In [25]:
#narrow down to only 5th class routes
df_final = df_final[(df_final['grade.YDS'].notna()) & (df_final['grade.YDS'].str.contains("5\."))]

#reduce grade
df_final['grade_reduced'] = df_final['grade.YDS'].apply(fully_reduce_grade)

This reduction greatly simplified the problem at hand, but I wanted to keep track of +/- to have the potential to reintroduce complexity. 

In [27]:
#create plus_minus secondary target column
df_final['plus_minus'] = df_final["grade.YDS"].apply(lambda x: 1 if x[-1] == '+' else (-1 if x[-1] == '-' else 0))

### Pulling Year out of FA
Currently the FA (first ascent, or establishment team of the route) is stored as a freeform string, although generally the names of the first ascentionists is listed first, followed by the year. There is also sometimes the FFA, or First Female Ascent. Because there are so many possible unique first ascentionists (not even including FA teams) my first goal is to pull out the year, and potentially pull out names later if I think it will truly help the model. For FA to be more useful, I believe that a smaller subset of known route establishers would have to be compiled, using deeper subject-area knowledge than I have.

In [66]:
df_final['fa'].head()

0             Jason Milford/ Matt Schutz Spring 2020
1                                       "Unknown" or
3    Wade Griffith, Sterling Killion, Scott Williams
4                                    Kroll and McHam
5                                      Bryan Carroll
Name: fa, dtype: object

In [61]:
#https://stackoverflow.com/questions/3276180/extracting-date-from-a-string-in-python
def extract_year(fa_string):
    matches = list(datefinder.find_dates(fa_string))

    if matches:
        # date returned will be a datetime.datetime object. here we are only using the first match.
        date = matches[0]
        year = date.year
        return year
    
    return np.nan

df_final['year_established'] = df_final['fa'].apply(extract_year)

In [62]:
df_final['year_established'].notna().sum()

22467

In [67]:
df_final['year_established'].head()

0   NaN
1   NaN
3   NaN
4   NaN
5   NaN
Name: year_established, dtype: float64

A quick visual inspection shows that this captured many years, but I do see an example where there is a standalone year in row 0 that did not get captured. We will get this one with regex.

In [68]:
def regex_year(fa_string, year_established):
    if year_established and not np.isnan(year_established):
        return year_established
    match = re.search(r'\d{4}', fa_string)
    if match:
        year = match.group()
        return int(year)
    return np.nan

df_final['year_established'] = df_final.apply(lambda x: regex_year(x['fa'], x['year_established']), axis=1)

In [69]:
df_final['year_established'].notna().sum()

43907

In [70]:
df_final['year_established'].head()

0    2020.0
1       NaN
3       NaN
4       NaN
5       NaN
Name: year_established, dtype: float64

Now we have all the standalone years captured as well! Although only about a third of the rows had the year established available, this is still potentially a feature for modeling. I'm going to do one last check for any outliers

In [71]:
df_final['year_established'].describe()

count    43907.000000
mean      1997.912861
std         42.562575
min          3.000000
25%       1988.000000
50%       2000.000000
75%       2013.000000
max       2996.000000
Name: year_established, dtype: float64

There do appear to be outliers in the year established we've extracted. I'm choosing to inspect/clean any years below 1850 and above 2022.

In [73]:
#checking early years
df_final[df_final['year_established'] < 1850]['fa']

9644           This guy and Gabriel L, or someone 1,500 years ago or those ants. Probably the ants.
21192                                   Sean Jones, Jake Jones [or Robbie Borchard 2/1005?], 9/2004
33591                                                                Jeff Foote, Steve Roper 11/192
42866                                                                       John Thomas, August 201
83426                                                      Mike Varlotta & Dave Gibson, October 200
91513                     Swartling & Mayer p.164, route not described but near  13 on diagram 36E.
91609                                                        near Swartling & Mayer p.164, route 17
91611                                                             Swartling & Mayer p.164, route 23
91612                                                             Swartling & Mayer p.164, route 22
91613                                                             Swartling & Mayer p.164, route 13


In [75]:
#found on MP
df_final.loc[21192, 'year_established'] = 2004

#http://indy-adventures.net/climbing/climb-maryland/TheNarrows.html
df_final.loc[83426, 'year_established'] = 2002

#https://pizbube.ch/wp-content/uploads/attachments/a_001/06B40F8DE9063EDAD8A99582C63591B3.pdf
df_final.loc[99582, 'year_established'] = 1957

#was just listed later in the fa
df_final.loc[105735, 'year_established'] = 1983

#https://americanalpineclub.org/news/2022/2/10/gunks-campground-amp-climbing-history
df_final.loc[106105, 'year_established'] = 1961

#just mistyped
df_final.loc[151860, 'year_established'] = 2003
df_final.loc[154363, 'year_established'] = 1995
df_final.loc[180013, 'year_established'] = 2017
df_final.loc[184074, 'year_established'] = 2009
df_final.loc[202826, 'year_established'] = 2018

#take care of the rest: just need to be nan. 1830-something was the lowest legitimate looking date
years_below_1830 = df_final[df_final['year_established'] < 1830].index
df_final.loc[years_below_1830, 'year_established'] = np.nan

In [78]:
#checking years beyond the current year
df_final[df_final['year_established'] > 2022][['fa', 'year_established']]

,fa,year_established
954,"John Ketcher, Richard Goodsen, Gary Goodsen, 3/70",2070.0
2087,"John Mendenhall's half-cyborg son XXXPLRRPRPT 66 and his buddy, Clyde.",2066.0
9637,"Chris C. and Taco, or perhaps it was 47 clowns, or everyone in Nickelback.",2047.0
13260,"Larry Reynolds & R. Wendell, 9/69, FFA: John Long, Rick Accomazzo, Richard Harrison & Tobin Sor...",2069.0
15247,"FA: Robbins, Wilson 7/53. FFA: Long, Sorenson, Harrison, Antel 1973.",2053.0
17956,Verne Clevenger and Bruce Chimilesk 7/71,2071.0
18181,"Wally Reed, Chuck Pratt 8/58, FFA Steve Roper and Mark Powell 1962",2058.0
18224,Tom Higgins and Bob Kamps 6/68,2068.0
18542,Tom Higgins and Tom Gerughty 8/70,2070.0
19686,"Tom Higgins, Ben Borrison, 8/68",2068.0


It appears that the date parser parsed many MM/DD/YY dates as being in the 2000s rather than the 1900s or below. This can be solved by simply subtracting 100 years from the date to bring it to the right century.

In [79]:
years_above_22 = df_final[df_final['year_established'] > 2022].index
df_final.loc[years_above_22, 'year_established'] = df_final.loc[years_above_22, 'year_established'] - 100

In [81]:
#sanity check
df_final[df_final['year_established'] > 2022][['fa', 'year_established']]

,fa,year_established
54017,"Josh Thurston, Jordan Garvey 2996",2896.0
64822,"Tim Bova and Mike Gray, 11/282020",2720.0


In [82]:
#final outliers
df_final.loc[54017, 'year_established'] = 1996
df_final.loc[64822, 'year_established'] = 2020

I realized in the process of moving the above cells from a later notebook to this one that this method of changing values is not very robust. The indices had to be changed between the notebooks, and any changes to the earlier portion of this notebook would also require the indices be changed again. For now, it works, but definitely a piece of the architecture to change in the future.

### Train/Validation/Test Split
I wanted to do the dataset split now so that consistent portions of the data are used for each model, and the splitting doesn't have to be repeated in each modeling notebook. Because there are so few rows, I am going with a smaller test size and a more regular validation size, stratified on both the reduced grade and +/- so we are able to introduce +/- complexity in the future.

In [83]:
#engineer stratify column
df_final['stratify'] = df_final['grade_reduced'].astype(str) + df_final['plus_minus'].astype(str)

#replace 5.151 and 5.15-1 with 5.150, since those only have one each in their class
index1 = df_final[df_final['stratify'] == "151"].index[0]
indexminus1 = df_final[df_final['stratify'] == '15-1'].index[0]
df_final.loc[[index1, indexminus1], 'stratify'] = '150'

In [90]:
train, test = train_test_split(df_final, test_size=.1, random_state=42, stratify=df_final['stratify'])
train, validation = train_test_split(train, test_size=.2, random_state=42, stratify=train['stratify'])

df_final.shape, train.shape, validation.shape, test.shape

((127008, 25), (91445, 25), (22862, 25), (12701, 25))

In [92]:
#save to csv
df_final.to_csv("./data/routes.csv", index=False)
train.to_csv("./data/train.csv", index=False)
validation.to_csv("./data/val.csv", index=False)
test.to_csv("./data/test.csv", index=False)